In [1]:
%pip install langsmith python-dotenv langchain langchain-openai langchain-pinecone


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
!pip install -U langsmith openevals openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.1/74.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [42]:
from langsmith import Client
import os
from langsmith import Client, wrappers
from openevals.llm import create_llm_as_judge
from openevals.prompts import CORRECTNESS_PROMPT
from openai import OpenAI

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_API_KEY"] = "~~~~~~~~~" # 실제 api key로 바꾸세용

client = Client()

dataset_name = "KorQuAD_dataset"

if not client.has_dataset(dataset_name=dataset_name):
    dataset = client.create_dataset(dataset_name)
else:
    dataset = client.read_dataset(dataset_name=dataset_name)

# korsquad 데이터셋을 이용해서 평가용 데이터셋 만들기

In [16]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 75.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 127.5 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 62.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstall

In [39]:
from datasets import load_dataset

# 1) 데이터셋 로드
korquad = load_dataset("squad_kor_v1")

In [40]:
validation_set = korquad["validation"]

examples = []
for sample in validation_set:
    question = sample["question"]
    texts    = sample["answers"]["text"]
    context = sample['context']
    answer = texts[0] if texts else ""
    examples.append({
            "inputs":  {"question": question},
            "outputs": {"answer":   answer},
            "metadata": {"contexts": context},
    })
# for ex in examples[:3]:
#     print(ex)

{'inputs': {'question': '임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은?'}, 'outputs': {'answer': '1989년 2월 15일'}, 'metadata': {'contexts': '1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의(폭력행위등처벌에관한법률위반)으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일~20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관 건물 계단을 내려오는 임종석을 발견, 검거해 구속을 집행했다. 임종석은 청량리경찰서에서 약 1시간 동안 조사를 받은 뒤 오전 9시 50분 경 서울 장안동의 서울지방경찰청 공안분실로 인계되었다.'}}
{'inputs': {'question': '1989년 6월 30일 평양축전에 대표로 파견 된 인물은?'}, 'outputs': {'answer': '임수경'}, 'metadata': {'contexts': '1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의(폭력행위등처벌에관한법률위반)으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일~20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 

In [43]:
client.create_examples(dataset_id=dataset.id, examples=examples)

{'example_ids': ['554b1866-8c2f-42c1-8087-27b3ad94541d',
  'dbbb317b-872a-4d75-bc60-d03076a4901f',
  'fa0167a2-258b-4f0e-9a10-0d3f0c62af98',
  '721edf1c-81fd-4c44-8176-e9871509c5ea',
  'def23f78-bafe-4366-831b-6b9d9ef3a59d',
  'df212d83-5c27-4183-bfef-504b0ac42bd7',
  'a83b1d7e-c46f-4aae-8a28-4bb75d740202',
  'fc1a27a9-1f7d-4407-adb8-8e06f13cdedf',
  '570160da-c4d5-4a90-990e-fbfdfcf7500a',
  '7c0bf31b-a852-4b32-9517-856767253248',
  '9d7a41ee-26b4-4454-9fb1-e75ad4b2aa2a',
  'f7fec2e9-b85e-4b5b-8c5a-1388c0d80d8d',
  'a83b87b7-7cb9-4356-9221-6e4b4c1cd23f',
  'e8d2eb42-3be3-400c-84e1-671c4258c522',
  'c73468b5-d923-4c95-80e7-f077a8dea82a',
  'c584cb40-28de-4be5-8455-4d7b8e5ec567',
  'c3ad7db9-bfca-48e9-a293-798ca32d2cfc',
  'bc0051cd-9474-4f21-8b2b-d0f34f311680',
  'b7778c22-c207-4e31-8e22-bd209553f590',
  '03885f7e-e968-46c8-ac65-d132f9fc777f',
  '8cb87fcf-f5d3-41a8-acc7-04a5dfa67e11',
  '428dea8d-6b5d-412f-8402-897e27d3f2bf',
  '32398026-db4f-4ad5-b936-bfffa49ac0d1',
  'ba948d78-a697-46

In [44]:
!pip install langchain-pinecone


[notice] A new release of pip is available: 23.3.1 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip


In [50]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

embeddings = HuggingFaceEmbeddings(model_name='nlpai-lab/KURE-v1')

pinecone_api_key = os.environ.get("PINECONE_API_KEY")
index_name="~~~~~~~~" # 업로드용으로 ~~ 처리했음
pc = Pinecone(api_key=pinecone_api_key)
vectorstore = PineconeVectorStore.from_existing_index(index_name=index_name,
                                                                   embedding=embeddings)
retriever = database.as_retriever()

OutOfMemoryError: CUDA out of memory. Tried to allocate 978.00 MiB. GPU 0 has a total capacty of 15.63 GiB of which 643.75 MiB is free. Process 3130000 has 14.99 GiB memory in use. Of the allocated memory 14.81 GiB is allocated by PyTorch, and 28.43 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

## LLM 답변 생성을 위한 RagBot
TODO: ragbot로 답변 생성하고 evaluator(openai 모델)로 답변 성능과 할루시네이션 측정 코드 작성하기! 

ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 